### AI프로젝트입문 (ECE30007-02)
Spring, 2023

Team Project: 미드바르 뿌리 이미지 인식 챌린지

`midbarClassification.py`

Author: 이현서 (ID#22100600) <hslee@handong.ac.kr>


### Modules

In [88]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image as Image

### Helper Functions

In [89]:
def imshow(img):
    img = img / 2 + 0.5 # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

### Train

In [90]:
EPOCHS = 50
BATCH_SIZES = 4
LEARNING_RATE = 0.0001

CLASSES = ["0012", "1224", "2436", "3648", "4860", "6072", "OVER"]
NUM_CLASSES = len(CLASSES)

DATASET_PATH = "./Dataset"
TRAIN_PATH = os.path.join(DATASET_PATH, "train")
TEST_PATH = os.path.join(DATASET_PATH, "test")

In [91]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [92]:
class midbarDataset(data.Dataset):
    def __init__(self, data_path, transform=None):
        self.data_path = data_path
        self.transform = transform
        self.image_paths, self.labels = self.load_data()

    def load_data(self):
        image_paths = []
        labels = []

        for class_name in CLASSES:
            class_dir = os.path.join(self.data_path, class_name)
            if os.path.exists(class_dir):
                class_images = [img for img in os.listdir(class_dir) if img.endswith(".png")]
                image_paths.extend([os.path.join(class_dir, img) for img in class_images])
                labels.extend([class_name] * len(class_images))

        return image_paths, labels

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]

        image = self.load_image(image_path)

        if self.transform:
            image = self.transform(image)

        return image, CLASSES.index(label)

    def load_image(self, image_path):
        image = Image.open(image_path).convert("L")
        image = image.resize((128, 128))
        return image

In [93]:
train_dataset = midbarDataset(TRAIN_PATH, transform=train_transform)
test_dataset = midbarDataset(TEST_PATH, transform=test_transform)

train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZES, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZES, shuffle=True)

In [94]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("GPU support : " + ("True" if torch.cuda.is_available() else "False"))

GPU support : False


In [95]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 7)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
net = Net().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

In [96]:
for epoch in range(EPOCHS):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), torch.tensor(data[1]).to(device)

        optimizer.zero_grad()
        outputs = Net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if i % 1000 == 999:
            print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print("Training finished.")

torch.save(net.state_dict(), "./midbar.pth")

/var/folders/3q/v_1rx_t54hq7jy00p987_tjw0000gn/T/ipykernel_36176/1366270942.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs, labels = data[0].to(device), torch.tensor(data[1]).to(device)


TypeError: Net.__init__() takes 1 positional argument but 2 were given

### Evaluation

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = Net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total * 100
print(f"Test Accuracy: {accuracy}%")